In [ ]:
from refinitiv.data.content import search
import pandas as pd
import numpy as np
import json
import sys
from enum import Enum

In [ ]:
class SearchBrowser:
    """The SearchBrowser is an interface allowing interrogation of the results of an RDP Search request.
       The goal of the interface is to help the user to determine the names of available Search properties
       and the values associated with them.  The interface provides methods to interrogate the details of
       a search to accelerate the ability to build proper search expressions.
       
       To start:
       
       1. Create a browser
          browser = SearchBrowser()
       2. Execute a search
          browser.execute(<criteria>)
       3. Interrogate the browser for details
          See below for interrogation methods.
    """
    # execute
    def execute(self, query=None, filter='', view=None, order_by='', navigators=''):
        """Apply a search expression to generate a hit containing values and attributes to browse.
           Returns: tuple (# of hits, # of properties generated from the first hit)
        """
        if view == None: 
            view = search.SearchViews.SEARCH_ALL
            
        # Retrieve metadata and prepare column details
        self.__meta = search.metadata.Definition(view=view).get_data()
        if not self.__meta.is_success:
            print(f'\nFailed to execute metadata extraction:\n{json.dumps(self.__meta.http_status, indent=2)}\n')
            raise NameError('Failed to execute search')             
        df = self.__meta.data.df
        
        # Search and process debug output
        debug = self.__search_and_extract(query, filter, view, order_by, navigators)

        if df.index.nlevels > 1:
            df.index.set_names(['Property', 'Nested'], inplace=True)
            df.reset_index(inplace=True)
            df.loc[(df.Property == df.Nested), 'Nested'] = ''
            self.__df = debug.join(df.set_index(['Property', 'Nested']), on=['Property', 'Nested'])
        else:
            debug.drop(columns=['Nested'], inplace=True)
            df.index.set_names(['Property'], inplace=True)
            df.reset_index(inplace=True)            
            self.__df = debug.join(df.set_index(['Property']), on='Property')
        
        self.df.replace({np.nan: ''}, inplace=True)
        return (self.hits, len(self.df))
    
    # values
    def __bool_values(self, text):
        df = self.type(self.PropertyType.Boolean)
        return df[df['Value']==text]
                  
    def values(self, text):
        """Browse the values that match the text expression.
           Eg: browser.values('united kingdom') - returns all values containing the expression 'united kingdom'
               browser.values(238)   - returns all numeric rows with an exact match
               browser.values(2.345)
        """
        if type(text) == str:
            return self.df.loc[self.df.Value.str.contains(text, na=False, case=False)]
        elif type(text) == bool:
            df = self.type(self.PropertyType.Boolean)
            return df[df['Value']==text]
        else:
            return self.df[self.df['Value']==text]

    # properties
    @property
    def hits(self):
        """# of hits based on the last search execution
        """        
        return self.__hits
    
    @property
    def df(self):
        """Lists the entire result table containing all properties, metadata, and their values, based on the 1st hit
        """        
        return self.__df
    
    @property
    def navigator(self):
        """If a navigator was specified in the request, this property presents a summary of the results
        """        
        return self.__navigator
    
    def properties(self, text):
        """Browse the properties that match the text expression.
           Eg: browser.properties('ISIN')
        """    
        return self.df.loc[self.df.Property.str.contains(text.replace(" ", ""), na=False, case=False)]

    # nested
    def nested(self, text):
        """Browse the nested properties that match the text expression.
           Eg: browser.nested('')
        """
        if 'Nested' in self.df:
            return self.df.loc[self.df.Nested.str.contains(text.replace(" ", ""), na=False, case=False)]

    # navigable
    def navigable(self, prop=None, value=None):
        """Browse the metadata that matches all properties that are navigable.
           Apply additional criteria that matches properties or values.
           Eg: browser.navigable()              - returns all navigable properties
               browser.navigable('Description') - returns all navigable properties containing 'Description'
               browser.navigable(value='euro')  - returns all navigable properties with a value containing 'euro'
               browser.navigable('RCS', 'euro') - returns all RCS-based navigable properties with a value containing 'euro'
        """
        return self.__interrogate(self.df[self.df['Navigable'] == True], prop, value)
       
    # exact
    def exact(self, prop=None, value=None):
        """Browse the metadata that matches all properties that provide an exact match filter expression.
           Apply an additional critera that matches properties.
           Eg: browser.exact()                - returns all exact properties
               browser.exact('Ticker')        - returns all exact properties containing 'Ticker'
               browser.exact('value=IBM')     - returns all exact properties with a value containing 'IBM'
               browser.exact('Ticker', 'IBM') - returns all exact Ticker-based exact properties with a value containing 'IBM'
        """
        return self.__interrogate(self.df[self.df['Exact'] == True], prop, value)
    
    # type
    def type(self, property_type):
        """Browse the types that match the specified property type.
           Eg: browser.type(SearchBrowser.PropertyType.Double) - returns all properties that have a double type        
        """
        if not isinstance(property_type, SearchBrowser.PropertyType):
            raise NameError(f'**Invalid property type specified.  Type must be: {SearchBrowser.PropertyType}\n\tEg: SearchBrowser.type(SearchBrowser.PropertyType.Double)')
        return self.df.loc[self.df.Type.str.contains(property_type.name, na=False, case=False)]    
    
    class PropertyType(Enum):
        Double = 1,
        String = 2,
        Date = 3,
        Boolean = 4,
        Integer = 5
    
    # __search_and_extract
    # Extracts the output from a _debugall request and organizes the results within a datafrmae
    def __search_and_extract(self, query, filter, view, order_by, navigators):
        data = []
               
        # Search
        response = search.Definition(
            view = view,
            query = query,
            filter = filter,
            top = 1,
            select = "_debugall",
            order_by = order_by,
            navigators = navigators
        ).get_data()
    
        if (response.http_status['http_status_code'] == 200):
            self.__hits = response.total
            
            # If available, process Navigator output
            self.__extract_navigator(response)
            
            if (response.total > 0):
                for prop, val in response.data.raw['Hits'][0]['raw_source'].items():
                    if (isinstance(val,list) and len(val) > 0 and isinstance(val[0], dict)):
                        for node in val:
                            for nested_prop, nested_val in node.items():
                                data.append([prop, nested_prop, nested_val])
                    else:
                        data.append([prop, '', val])
            return pd.DataFrame(data, columns=['Property', 'Nested', 'Value'])
            
        else:
            print(f'\nFailed to execute search:\n{json.dumps(response.http_status, indent=2)}\n')
            raise NameError('Failed to execute search')         
            
    # __extract_navigators
    # If present, extracts the navigator details
    def __extract_navigator(self, response):
        table = {}
        
        if "Navigators" in response.data.raw:
            navigators = list(response.data.raw['Navigators'].items())
            
            base = None
            for navigator in navigators:
                for key, bucket in navigator[1].items():
                    counts = []
                    columns = []
                    filters = []
                    
                    for val in bucket:
                        counts.append(val['Count'] if 'Count' in val else 0)
                        columns.append(val['Label'] if 'Label' in val else 'NA')
                        
                        if ('Filter' in val):
                            filters.append(val['Filter'])

                    if (base is None):
                        base = counts
                    if (base == counts):
                        table[navigator[0]] = columns
                        if len(filters) > 0:
                            table['Filter'] = filters
                    else:
                        print(f'Ignoring Navigator: {navigator[0]} - mismatched.', file = sys.stderr )
                        
            table["Count"] = base
            self.__navigator = pd.DataFrame(table)
            
    # __interrogate
    # Interrogate the dataframe for properties and values
    def __interrogate(self, df, prop, value):
        if prop != None and value != None:
            prop = df.loc[df.Property.str.contains(prop, na=False, case=False)]
            return prop.loc[prop.Value.str.contains(value, na=False, case=False)]
        if prop != None:
            return df.loc[df.Property.str.contains(prop, na=False, case=False)]
        elif value != None:
            return df.loc[df.Value.str.contains(value, na=False, case=False)]
        else:
            return df        
        
    # Instantiate a SearchBrowser object
    def __init__(self):        
        self.__df = {}
        self.__meta = {}
        self.__navigator = {}
        self.__hits = 0